<a href="https://colab.research.google.com/github/Keer-Luo/ISYS5002-Assignment/blob/main/Sandbox_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment Ideas

1. State the problem clearly
2. Describle the input and output information
3. Work the problem
4. Develop an algorithm
5. Test solution with a variety of data

# Inputs

* input a ticker symbol. Not sure what a ticker symbol is?

Note: To prompt the user, I guess I will need to use the input() function

**TODO: Find out what a ticker symbol is** 

# Processing thoughts

* Scrap the Yahoo website
* Extract the 'Key Executives'
* Calculate income tax

Note: In a previous Notebook, we calculated the tax based on ATO table, Might be able to  use that in this assignment

**TODO: Find Yahoo Website**

Here is the webpage: https://finance.yahoo.com/

Note there is a search bar, will use to find a company

**TODO: Find the key Executivies**
# Outputs

* Save the CSV files
* save to SQL database 

**TODO: Work out what I have to save!**


# Lets Explore the Website

1. Search for Coles

Coles ticker is COL.AX

Note the 'AX' means Australian

Just read the assignment specification, it can be any company.

Found the key executivies in the profile page.

Here is the URL for Coles:

https://finance.yahoo.com/quote/COL.AX/profile?p=COL.AX

2. Lets search for National Australian Back

The ticker is NAB.AX, Here is the URL to the proflie page:

https://finance.yahoo.com/quote/NAB.AX/profile?p=NAB.AX

There appears to be a pattern with the URL. I can see that the ticker twice in the URL

Manual process is
1. Seaech for Ticker
2. Click on proflie

**TODO: Can I shortcut the process and just use the URL**

Looks like I can edit the URL and replace the ticker. Seems that only the first ticker symbol in the URL is important, but to be safe I will replace both.




In [ ]:
#install requirements
!pip install kora -q

#load package page
from bs4 import BeautifulSoup
from kora.selenium import wd

     |████████████████████████████████| 57 kB 2.6 MB/s 
     |████████████████████████████████| 56 kB 3.4 MB/s 


In [ ]:
https://finance.yahoo.com/

SyntaxError: ignored